Dataset Link: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [27]:
def read_df(filename):
    df = pd.read_parquet(filename)
    print("number of columns in the dataset: "+str(len(df.columns)))
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

    df['duration']=df.tpep_dropoff_datetime - df.tpep_pickup_datetime 

    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    print("duration std "+str(df.duration.std()))
    before= df.shape[0]
    print("before removing outliers: "+str(before ))
    df= df[((df.duration>=1) &(df.duration<=60))]
    after = df.shape[0]
    print("after removing outliers: "+ str(after))
    print("ratio: " + str(after/before))
    categorical=['PULocationID', 'DOLocationID']

    df[categorical]=df[categorical].astype(str)
    return df

In [28]:
filename_train ='./data/yellow_tripdata_2022-01.parquet'
df_train = read_df(filename_train)

number of columns in the dataset: 19
duration std 46.44530513776499
before removing outliers: 2463931
after removing outliers: 2421440
ratio: 0.9827547930522406


In [29]:
filename_val = './data/yellow_tripdata_2022-02.parquet'
df_val = read_df(filename_val)

number of columns in the dataset: 19
duration std 47.26394076781901
before removing outliers: 2979431
after removing outliers: 2918187
ratio: 0.979444397269143


In [19]:
dv = DictVectorizer()

categorical= ['PULocationID', 'DOLocationID']

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [22]:
y_train = df_train['duration'].values

In [25]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_val['duration'].values

In [23]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190135969309

In [21]:
len(dv.feature_names_)

515

In [26]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.786389386061076